---
title: "Chapter BA10A: Probability of a Hidden Path"
format:
  html:
    code-fold: false
    toc: true
jupyter: python3
---

## Problem Statement and Biological Context

**Given:** A hidden path π followed by the states States and transition matrix Transition of an HMM (Σ, States, Transition, Emission).

**Return:** The probability of this path, $Pr(\pi)$. You may assume that initial probabilities are equal.


**Sample Dataset:**
```
AABBBAABABAAAABBBBAABBABABBBAABBAAAABABAABBABABBAB
--------
A   B
--------
    A   B
A   0.194   0.806
B   0.273   0.727
```

**Sample Output:**
```
5.01732865318e-19
```

This problem introduces us to Hidden Markov Models (HMMs), one of the most fundamental statistical frameworks in bioinformatics[1][2]. HMMs are probabilistic models that describe sequences where the underlying process generating the observations is hidden from direct view[2][8]. In biological contexts, HMMs are extensively used for gene finding, protein structure prediction, sequence alignment, and identifying functional domains in proteins[8][29].

The hidden path probability calculation forms the foundation for more complex HMM algorithms like the Viterbi algorithm for decoding and the Forward-Backward algorithm for parameter estimation[5][6]. Understanding this computation is essential because it represents the core mathematical operation that quantifies how likely a particular sequence of hidden states is given the model parameters.

In genomics, this could model scenarios such as CG-island detection (where hidden states represent "inside" vs "outside" CG-rich regions), gene structure annotation (exons vs introns), or chromatin state segmentation (active vs inactive transcription regions)[1][42]. The transition probabilities capture the biological tendency for these states to persist or change over genomic coordinates.

## Mathematical Foundation

### Hidden Markov Model Definition

A Hidden Markov Model is formally defined as a tuple $$ M = (Q, \Sigma, A, E, \pi) $$ where[2][8]:

- $( Q = \{q_1, q_2, \ldots, q_N\} )$ is the set of hidden states
- $( \Sigma )$ is the observation alphabet  
- $( A )$ is the $( N \times N )$ state transition matrix
- $( E )$ is the $( N \times |\Sigma| )$ emission matrix
- $( \pi )$ is the initial state probability distribution

For this problem, we focus on the transition matrix $( A )$ where $$ A_{ij} = P(q_{t+1} = j \mid q_t = i) $$ represents the probability of transitioning from state $( i )$ to state $( j )$.

### Hidden Path Probability Calculation

Given a hidden path $( \pi = \pi_1\pi_2\ldots\pi_L )$ of length $( L )$, the probability of this path is computed using the Markov property[2][5]:

$$P(\pi) = P(\pi_1) \prod_{t=2}^{L} P(\pi_t | \pi_{t-1})$$

Under the equal initial probability assumption, $P(\pi_1) = \frac{1}{|Q|}$, so:

$$P(\pi) = \frac{1}{|Q|} \prod_{t=2}^{L} A_{\pi_{t-1}, \pi_t}$$

### Numerical Stability Considerations

For long sequences, the probability product can become extremely small, leading to numerical underflow[40][43]. The standard approach is to work in log space:

$$\log P(\pi) = \log\left(\frac{1}{|Q|}\right) + \sum_{t=2}^{L} \log A_{\pi_{t-1},\pi_t}$$

This transforms the product into a sum, making computations numerically stable and more efficient[43][45].

### Complexity Analysis

The time complexity is $( O(L) )$ where $( L )$ is the path length, since we perform one transition probability lookup per position. Space complexity is $( O(1) )$ for the computation itself, plus $( O(N^2) )$ to store the transition matrix where $( N )$ is the number of states.

## Reference Implementation

In [1]:
import numpy as np
import numba
from typing import List, Dict, Tuple
from numba import njit

In [2]:
def parse_hmm_input(input_text: str) -> Tuple[str, List[str], np.ndarray]:
    """
    Parse HMM input format from Rosalind problem.
    
    Args:
        input_text: Raw input string containing path, states, and transition matrix
        
    Returns:
        Tuple of (hidden_path, states_list, transition_matrix)
    """
    lines = [line.strip() for line in input_text.strip().split('\n') if line.strip()]
    
    # Extract hidden path
    hidden_path = lines[0]
    
    # Find the SECOND separator line (the one before the matrix header)
    separator_indices = []
    for i, line in enumerate(lines):
        if '---' in line:
            separator_indices.append(i)
    
    if len(separator_indices) < 2:
        raise ValueError("Invalid input format: need two separator lines")
    
    # Use the first separator to find states
    first_separator = separator_indices[0]
    states_line = lines[first_separator + 1]
    states = states_line.split()
    n_states = len(states)
    
    # Use the second separator to find matrix start
    second_separator = separator_indices[1]
    
    # Create state to index mapping
    state_to_idx = {state: i for i, state in enumerate(states)}
    
    # Parse transition matrix
    transition_matrix = np.zeros((n_states, n_states), dtype=np.float64)
    
    # Matrix starts after second separator + header row
    matrix_start = second_separator + 2  # Skip separator and header row
    
    # Parse each row of the transition matrix
    for i in range(n_states):
        if matrix_start + i >= len(lines):
            raise ValueError("Invalid input format: transition matrix incomplete")
            
        row_data = lines[matrix_start + i].split()
        
        # First element should be the from-state
        from_state = row_data[0]
        if from_state not in state_to_idx:
            raise ValueError(f"Unknown state: {from_state}")
        
        from_idx = state_to_idx[from_state]
        
        # Remaining elements are the probabilities
        if len(row_data) != n_states + 1:  # +1 for the state name
            raise ValueError(f"Invalid row format for state {from_state}: expected {n_states + 1} elements, got {len(row_data)}")
        
        for j in range(n_states):
            try:
                prob = float(row_data[j + 1])  # +1 to skip the state name
                transition_matrix[from_idx, j] = prob
            except ValueError as e:
                raise ValueError(f"Invalid probability value '{row_data[j + 1]}' in row for state {from_state}")
    
    return hidden_path, states, transition_matrix


In [3]:
def encode_path(hidden_path: str, states: List[str]) -> np.ndarray:
    """
    Convert symbolic path to integer array for efficient computation.
    
    Args:
        hidden_path: String of state symbols
        states: List of valid states
        
    Returns:
        Integer array representing the path
    """
    state_to_idx = {state: i for i, state in enumerate(states)}
    return np.array([state_to_idx[char] for char in hidden_path], dtype=np.int32)

In [4]:
@njit
def compute_path_probability_log(encoded_path: np.ndarray, 
                                transition_matrix: np.ndarray) -> float:
    """
    Compute log probability of hidden path using Numba optimization.
    
    This function is optimized for numerical stability by working in log space
    and uses Numba JIT compilation for performance.
    
    Args:
        encoded_path: Integer-encoded hidden path
        transition_matrix: Transition probability matrix
        
    Returns:
        Log probability of the path
    """
    if len(encoded_path) == 0:
        return 0.0
    
    n_states = transition_matrix.shape[0]
    
    # Equal initial probability: log(1/n_states)
    log_prob = -np.log(n_states)
    
    # Add transition probabilities
    for t in range(1, len(encoded_path)):
        prev_state = encoded_path[t-1]
        curr_state = encoded_path[t]
        trans_prob = transition_matrix[prev_state, curr_state]
        
        # Add small epsilon to prevent log(0)
        if trans_prob <= 0:
            return -np.inf
        
        log_prob += np.log(trans_prob)
    
    return log_prob

In [5]:
def compute_hidden_path_probability(hidden_path: str, 
                                   states: List[str], 
                                   transition_matrix: np.ndarray) -> float:
    """
    Main function to compute hidden path probability.
    
    Args:
        hidden_path: String representing the hidden state sequence
        states: List of state labels
        transition_matrix: N×N transition probability matrix
        
    Returns:
        Probability of the hidden path
    """
    # Validate inputs
    if len(hidden_path) == 0:
        return 1.0
    
    # Check if all characters in path are valid states
    valid_states = set(states)
    if not all(char in valid_states for char in hidden_path):
        raise ValueError("Hidden path contains invalid states")
    
    # Verify transition matrix properties
    if not np.allclose(transition_matrix.sum(axis=1), 1.0, rtol=1e-10):
        raise ValueError("Transition matrix rows must sum to 1")
    
    # Encode path for efficient computation
    encoded_path = encode_path(hidden_path, states)
    
    # Compute log probability for numerical stability
    log_prob = compute_path_probability_log(encoded_path, transition_matrix)
    
    # Convert back to regular probability
    return np.exp(log_prob)

### Example usage with sample data

In [6]:

def solve_rosalind_ba10a(input_text: str) -> str:
    """
    Solve the Rosalind BA10A problem.
    
    Args:
        input_text: Problem input in Rosalind format
        
    Returns:
        Formatted probability result
    """
    hidden_path, states, transition_matrix = parse_hmm_input(input_text)
    probability = compute_hidden_path_probability(hidden_path, states, transition_matrix)
    
    # Format to match expected output precision
    return f"{probability:.11e}"

### Sample problem execution

In [7]:
sample_input = """AABBBAABABAAAABBBBAABBABABBBAABBAAAABABAABBABABBAB
--------
A   B
--------
    A   B
A   0.194   0.806
B   0.273   0.727"""
print(sample_input)
result = solve_rosalind_ba10a(sample_input)
print(f"Hidden path probability: {result}")

AABBBAABABAAAABBBBAABBABABBBAABBAAAABABAABBABABBAB
--------
A   B
--------
    A   B
A   0.194   0.806
B   0.273   0.727
Hidden path probability: 5.01732865318e-19


## Interactive Exploration

In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [15]:
def create_interactive_hmm_explorer():
    """
    Create interactive widgets to explore HMM path probability calculations.
    """
    
    # Widget controls
    path_input = widgets.Text(
        value='AABBBAABABAAAABBBBAABBABABBBAABBAAAABABAABBABABBAB',
        placeholder='Enter hidden path...',
        description='Hidden Path:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    prob_aa = widgets.FloatSlider(
        value=0.194,
        min=0.001,
        max=0.999,
        step=0.001,
        description='P(A→A):',
        style={'description_width': 'initial'}
    )
    
    prob_ab = widgets.FloatSlider(
        value=0.806,
        min=0.001,
        max=0.999,
        step=0.001,
        description='P(A→B):',
        style={'description_width': 'initial'}
    )
    
    prob_ba = widgets.FloatSlider(
        value=0.273,
        min=0.001,
        max=0.999,
        step=0.001,
        description='P(B→A):',
        style={'description_width': 'initial'}
    )
    
    prob_bb = widgets.FloatSlider(
        value=0.727,
        min=0.001,
        max=0.999,
        step=0.001,
        description='P(B→B):',
        style={'description_width': 'initial'}
    )
    
    output = widgets.Output()
    
    # Flag to prevent recursive updates
    _updating = False
    
    def update_calculation(*args):
        """Update probability calculation based on widget values."""
        nonlocal _updating
        
        if _updating:  # Prevent recursive calls
            return
            
        _updating = True
        
        try:
            with output:
                clear_output(wait=True)
                
                # Normalize probabilities to ensure rows sum to 1
                sum_a = prob_aa.value + prob_ab.value
                sum_b = prob_ba.value + prob_bb.value
                
                norm_aa = prob_aa.value / sum_a
                norm_ab = prob_ab.value / sum_a
                norm_ba = prob_ba.value / sum_b
                norm_bb = prob_bb.value / sum_b
                
                # Create transition matrix with normalized values
                transition_matrix = np.array([
                    [norm_aa, norm_ab],
                    [norm_ba, norm_bb]
                ])
                
                states = ['A', 'B']
                hidden_path = path_input.value
                
                try:
                    # Calculate probability
                    probability = compute_hidden_path_probability(hidden_path, states, transition_matrix)
                    
                    # Display results
                    print(f"Hidden Path: {hidden_path}")
                    print(f"Path Length: {len(hidden_path)}")
                    print(f"\nTransition Matrix (normalized):")
                    print(f"     A      B")
                    print(f"A  {norm_aa:.3f}  {norm_ab:.3f}")
                    print(f"B  {norm_ba:.3f}  {norm_bb:.3f}")
                    
                    # Show normalization info if values were adjusted
                    if abs(sum_a - 1.0) > 1e-6 or abs(sum_b - 1.0) > 1e-6:
                        print(f"\nOriginal values (before normalization):")
                        print(f"Row A sum: {sum_a:.6f} -> normalized to 1.0")
                        print(f"Row B sum: {sum_b:.6f} -> normalized to 1.0")
                    
                    print(f"\nPath Probability: {probability:.6e}")
                    print(f"Log Probability: {np.log(probability):.6f}")
                    
                    # Count transitions
                    transitions = {'AA': 0, 'AB': 0, 'BA': 0, 'BB': 0}
                    for i in range(len(hidden_path) - 1):
                        trans = hidden_path[i] + hidden_path[i+1]
                        transitions[trans] += 1
                    
                    print(f"\nTransition Counts:")
                    for trans, count in transitions.items():
                        print(f"{trans}: {count}")

                    # Note about transition counts
                    print(f"\nNote: Transition counts are fixed for this sequence.")
                    print(f"They represent actual observed transitions in the hidden path.")
                    print(f"Only the probability calculation changes with different transition matrices.")
                    
                    # Show step-by-step calculation for short paths
                    if len(hidden_path) <= 10:
                        print(f"\nStep-by-step calculation:")
                        print(f"Initial probability: 1/{len(states)} = {1/len(states)}")
                        
                        log_prob = -np.log(len(states))
                        for i in range(1, len(hidden_path)):
                            prev_state = hidden_path[i-1]
                            curr_state = hidden_path[i]
                            prev_idx = states.index(prev_state)
                            curr_idx = states.index(curr_state)
                            trans_prob = transition_matrix[prev_idx, curr_idx]
                            log_prob += np.log(trans_prob)
                            print(f"Step {i}: {prev_state}→{curr_state}, P = {trans_prob:.6f}, Log P = {np.log(trans_prob):.6f}")
                        
                        print(f"Final log probability: {log_prob:.6f}")
                        print(f"Final probability: {np.exp(log_prob):.6e}")
                        
                except Exception as e:
                    print(f"Error: {str(e)}")
                    
        finally:
            _updating = False
    
    # Link widgets to update function
    path_input.observe(update_calculation, names='value')
    prob_aa.observe(update_calculation, names='value')
    prob_ab.observe(update_calculation, names='value')
    prob_ba.observe(update_calculation, names='value')
    prob_bb.observe(update_calculation, names='value')
    
    # Initial calculation
    update_calculation()
    
    # Display interface
    display(widgets.VBox([
        widgets.HTML("<h3>Interactive HMM Path Probability Explorer</h3>"),
        path_input,
        widgets.HTML("<h4>Transition Probabilities (automatically normalized)</h4>"),
        widgets.HTML("<p><em>Note: Values are automatically normalized so each row sums to 1.0</em></p>"),
        prob_aa,
        prob_ab,
        prob_ba, 
        prob_bb,
        output
    ]))

create_interactive_hmm_explorer()